In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle as pkl
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
from xarray import DataArray
from tqdm.auto import tqdm
from PIL import Image

import asyncio
from schema_agents.role import Role
from schema_agents.schema import Message
from pydantic import BaseModel, Field
from typing import List

from bioimageio_chatbot.image_processor import *


/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-08 10:48:49.993 | INFO     | schema_agents.config:__init__:44 - Config loading done.
2023-12-08 10:48:49.993 | INFO     | schema_agents.config:__init__:55 - Set OPENAI_API_BASE in case of network issues


In [3]:
class SearchAndRunModelByImage(BaseModel):
    """Search bioimage model zoo for the best model to analyze user's image and run the model according to user's request"""
    request: str = Field(description="The user's request")

class InputImageDescription(BaseModel):
    """"Some description of the input image"""
    image_shape: tuple = Field(description = "The shape of the input image.")
    # image_dtype: str = Field(description = "The dtype of the input image.")
    # image_filename: str = Field(description = "The filename of the input image.")

class InputImageAxes(BaseModel):
    """The axes of the input image"""
    axes: str = Field(description="The axes of the input image, for example 'cyx', 'cyxz', 'tczyx'...")

async def guess_image_axes(input_image_description : InputImageDescription, role: Role = None) -> InputImageAxes:
      """Guess the axis labels given the provided image description"""
      response = await role.aask(input_image_description, InputImageAxes)
      return(response)

In [5]:
async def main():
    db_path = "../tmp/image_db/"
    image_dir = "../tmp/image_db/input_images/"

    customer_service = Role(name = "Melman",
                    profile = "Customer Service",
                    goal="Your goal as Melman from Madagascar, the community knowledge base manager, is to assist users in effectively utilizing the knowledge base for bioimage analysis. You are responsible for answering user questions, providing clarifications, retrieving relevant documents, and executing scripts as needed. Your overarching objective is to make the user experience both educational and enjoyable.",
                constraints=None,
                actions=[guess_image_axes])
    event_bus = customer_service.get_event_bus()
    event_bus.register_default_events()
    input_images = [os.path.join(image_dir, x) for x in os.listdir(image_dir) if x.endswith('.npy')]
    image_processor = ImageProcessor()
    guessed_axes = []
    true_axes = []
    true_shapes = []
    for input_image_path in input_images:
        model_name = os.path.splitext(os.path.basename(input_image_path))[0]
        input_axes = get_axes(db_path, model_name)
        input_image = image_processor.read_image(input_image_path)
        desc = InputImageDescription(image_shape=input_image.shape)
        m = Message(content = 'guess the image axes', data = desc, role = 'User')
        responses = await customer_service.handle(m)
        guess = responses[0].data.axes
        guessed_axes.append(guess)
        true_axes.append(input_axes)
        true_shapes.append(input_image.shape)
    return((input_images, true_shapes, true_axes, guessed_axes))

loop = asyncio.get_event_loop()
f = await main()
# loop.create_task(main())
# loop.run_forever()
loop.run_until_complete(f)

2023-12-08 10:49:18.931 | INFO     | schema_agents.provider.openai_api:__init__:150 - OpenAI API model: gpt-4-1106-preview
2023-12-08 10:49:18.931 | INFO     | schema_agents.role:set_event_bus:83 - Mounting Melman(Customer Service) to event bus: Customer Service - Melman.



Generating InputImageAxes (start): 
{"axes":"cyx"}
Generating InputImageAxes (finished): {"axes":"cyx"}


2023-12-08 10:49:21.354 | INFO     | schema_agents.provider.openai_api:update_cost:101 - Total running cost: $0.003 | Max budget: $10.000 | Current cost: $0.003, prompt_tokens=236, completion_tokens=9



Generating InputImageAxes (start): 


2023-12-08 10:49:22.562 | INFO     | schema_agents.provider.openai_api:update_cost:101 - Total running cost: $0.005 | Max budget: $10.000 | Current cost: $0.003, prompt_tokens=238, completion_tokens=10


...

2023-12-08 10:49:32.836 | INFO     | schema_agents.provider.openai_api:update_cost:101 - Total running cost: $0.027 | Max budget: $10.000 | Current cost: $0.003, prompt_tokens=238, completion_tokens=9


x"}
Generating InputImageAxes (finished): {"axes":"cyx"}

Generating InputImageAxes (start): 
{"axes

2023-12-08 10:49:33.978 | INFO     | schema_agents.provider.openai_api:update_cost:101 - Total running cost: $0.029 | Max budget: $10.000 | Current cost: $0.003, prompt_tokens=236, completion_tokens=9


":"cyx"}
Generating InputImageAxes (finished): {"axes":"cyx"}


ReadTimeout: 

In [89]:
if loop.is_running():
    loop.close()

RuntimeError: Cannot close a running event loop

In [1]:
# loop.run_until_complete()
import asyncio
asyncio.get_event_loop()

<_UnixSelectorEventLoop running=True closed=False debug=False>

In [93]:
loop.clo

<_UnixSelectorEventLoop running=True closed=False debug=False>